In [1]:
import  sqlalchemy as sqla 
import os

In [2]:
! pip install psycopg2-binary

# Leer Variables de Ambiente

In [5]:
POSTGRES_DB = os.getenv('POSTGRES_DB')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')

# Funciones Auxiliares

In [14]:
from sqlalchemy import create_engine
# create a new engine
def start_engine(echo:bool=True):
    db_string = f'postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}'
    engine = create_engine(db_string, echo=echo)
    
    return engine

In [15]:
from sqlalchemy import inspect
def existe_tabla(nombre_tabla:str):
    engine = start_engine(echo=False)
    inspector = inspect(engine)
    schemas = inspector.get_schema_names()

    for schema in schemas:
        for table_name in inspector.get_table_names(schema=schema):
            if nombre_tabla == table_name:
                print(f"Table: {table_name} existe ")  
                print("Columns:  ")
                for column in inspector.get_columns(table_name, schema=schema):
                    print("Column: %s" % column)
                return True
    return False

In [27]:
from sqlalchemy.orm import sessionmaker
def create_session(echo:bool=False):
    engine = start_engine(echo=echo)
    Session = sessionmaker(bind=engine)
    session = Session()
    
    return session

In [24]:
from sqlalchemy import create_engine, MetaData, Table, select

def glipse_tabla(nombre_tabla:str, obs:int=5, engine=None):

    # Nuevo engine
    if not engine:
        engine = start_engine(echo=False)
    # Crear objeto metadata para reflejar la metadata existente en la tabla
    metadata = MetaData()
    students_table = Table(nombre_tabla, metadata,  autoload_with=engine)
    # Seleccionar solo las primeras 3 rows
    select_stmt = select(students_table).limit(obs)
    # Crear la coneccion y ejecutar el statement
    conn = engine.connect()
    result = conn.execute(select_stmt)
    # print the rows
    for row in result:
        print(row)
    # close the connection
    conn.close()

# Crear Nuevas Tablas

In [19]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [20]:
engine = start_engine(False)

# define the students table
Base = declarative_base()

class Clase1(Base):
    __tablename__ = 'Clase1'
    cu = Column(Integer, primary_key=True)
    grade = Column(Float)
        
    
Base.metadata.create_all(engine)


In [25]:
existe_tabla('Clase1')

Table: Clase1 existe 
Columns:  
Column: {'name': 'cu', 'type': INTEGER(), 'nullable': False, 'default': 'nextval(\'"public"."Clase1_cu_seq"\'::regclass)', 'autoincrement': True, 'comment': None}
Column: {'name': 'grade', 'type': DOUBLE_PRECISION(precision=53), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}


True

In [21]:
engine = start_engine(False)

# define the students table
Base = declarative_base()

class Clase2(Base):
    __tablename__ = 'Clase2'
    cu = Column(Integer, primary_key=True)
    grade = Column(Float)
    
Base.metadata.create_all(engine)


In [26]:
existe_tabla('Clase2')

Table: Clase2 existe 
Columns:  
Column: {'name': 'cu', 'type': INTEGER(), 'nullable': False, 'default': 'nextval(\'"public"."Clase2_cu_seq"\'::regclass)', 'autoincrement': True, 'comment': None}
Column: {'name': 'grade', 'type': DOUBLE_PRECISION(precision=53), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}


True

# Popular tablas

In [30]:
from sqlalchemy.orm import declarative_base


In [31]:
class Student(Base):
    __tablename__ = 'students'
    cu = Column(Integer, primary_key=True)
    nombre = Column(String)
    apellido = Column(String)
    grade = Column(Float)

In [44]:
import warnings


class ClaseWarnings(Warning):
    pass

class ExisteCU(ClaseWarnings):
    pass

class ModificarCU(ClaseWarnings):
    pass

In [80]:
class AddGrade():
    
    def __init__(self, table:declarative_base):
        self.table = table
        self.main_base = Student
    
    def revisar_integridad_base(self,cu:int):
        student = session.query(self.main_base).filter_by(cu=cu).first()
        if student == None:
            raise Exception(f'La cu {cu} no existe en la base principal')
        else:
            pass
    
    def revisar_integridad_tabla(self,cu:int, grade:float):
        
        assert(0.0 <= grade <= 10.0)
        assert(0 < cu)
        student = session.query(self.table).filter_by(cu=cu).first()
        if student == None: # Nuevx alumne y grade
            return True
        else:
            warnings.warn(f"Entrada con el \'cu\' {cu} ya existe, utiliza el metodo apropiado para modificarla", ExisteCU)
            return False
    
    def add_grade(self,cu:int, grade:float):
        self.revisar_integridad_base(cu)
        existe_grade = self.revisar_integridad_tabla(cu, grade)
        if existe_grade:
            session = create_session(False)
            # Crear una observacion o instancia de Student usando su clase
            calificacion = self.table(cu=cu, grade=grade)
            # Agregarla
            session.add(calificacion)
            # Commit a la base
            try:
                session.commit()
            except IntegrityError:
                session.rollback()
                print('IntegrityError: Duplicate key value')
        else:
            pass

            
            
        
        

In [84]:
clase1_manipulator = AddGrade(Clase1)

In [82]:
clase1_manipulator.add_grade(cu=1, grade=8.0)

/tmp/ipykernel_57/1918778077.py:22: ExisteCU: Entrada con el 'cu' 1 ya existe, utiliza el metodo apropiado para modificarla
  warnings.warn(f"Entrada con el \'cu\' {cu} ya existe, utiliza el metodo apropiado para modificarla", ExisteCU)


In [83]:
glipse_tabla(nombre_tabla='Clase1')

(1, 8.0)


In [85]:
clase1_manipulator.add_grade(cu=18, grade=8.0)

Exception: La cu 18 no existe en la base principal

**Proyecto**: Modificar la clase para que acepte listas de diccionarios. Ejemplo: Que pueda agregar a todes en la siguiente lista [{'cu':5, 'grade':9.3}, {'cu':6, 'grade':7.3}, ...]

In [86]:
clase2_manipulator = AddGrade(Clase2)

In [87]:
clase2_manipulator.add_grade(cu=2, grade=9.0)

In [88]:
glipse_tabla(nombre_tabla='Clase2')

(2, 9.0)
